In [13]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

coviddeathsdata = pd.read_csv('https://raw.githubusercontent.com/S-I-SVD/Randomized-SVD/main/data/covid%20data/covid_new_cases.csv')
coviddeathsdata = coviddeathsdata.drop(columns=['Unnamed: 0'])
coviddata_meta = coviddeathsdata.loc[:, ['state']]
coviddeathsdata = coviddeathsdata.drop(columns=['state'])
M = coviddeathsdata.values
columnmeans = np.array([np.mean(M[:, i]) for i in range (0,M.shape[1])])
columnmeansmatrix = columnmeans * np.ones(M.shape)
M_scaled = M - columnmeansmatrix
coviddata_scaled = M_scaled
fig = go.Figure(data=[go.Surface(z=coviddeathsdata.values)])
fig.update_layout(title='COVID-19 New Cases Per Day',autosize=True)
fig.show()

In [8]:
coviddata_scaled = coviddeathsdata-coviddeathsdata.mean()
u, s, v = np.linalg.svd(coviddata_scaled, full_matrices=True)
var_explained = np.round(s**2/np.sum(s**2), decimals=3)
labels= ['SV'+str(i) for i in range(1,51)]
svd_df = pd.DataFrame(u[:,0:50], index=coviddata_meta["state"].tolist(), columns=labels)
svd_df=svd_df.reset_index()
svd_df.rename(columns={'index':'state'}, inplace=True)
s = np.diag(s)
approx_uptorank1_meancentered = u[:,:1] @ s[0:1,:1] @ v[:1,:]
a = coviddeathsdata.mean()
b = a.values
approx_uptorank1 = approx_uptorank1_meancentered + b
fig = go.Figure(data=[go.Surface(z=approx_uptorank1)])
fig.update_layout(title='SV1 Only',autosize=True)
fig.show()

In [9]:
approx_uptorank2_meancentered = u[:,:2] @ s[0:2,:2] @ v[:2,:]
approx_justrank2 = approx_uptorank2_meancentered - approx_uptorank1_meancentered + b
fig = go.Figure(data=[go.Surface(z=approx_justrank2)])
fig.update_layout(title='SV2 Only',autosize=True)
fig.show()

In [10]:
approx_uptorank3_meancentered = u[:,:3] @ s[0:3,:3] @ v[:3,:]
approx_justrank3 = approx_uptorank3_meancentered - approx_uptorank2_meancentered - approx_uptorank1_meancentered + b
fig = go.Figure(data=[go.Surface(z=approx_justrank3)])
fig.update_layout(title='SV3 Only',autosize=True)
fig.show()

In [11]:
approx_uptorank3 = approx_uptorank3_meancentered + b
fig = go.Figure(data=[go.Surface(z=approx_uptorank3)])
fig.update_layout(title='Approximation Up To SV3',autosize=True)
fig.show()

In [12]:
covid = coviddeathsdata.values
residualnumbers = covid - approx_uptorank3
fig = go.Figure(data=[go.Surface(z=residualnumbers)])
fig.update_layout(title='Residual',autosize=True)
fig.show()